In [ ]:
!pip install transformers
!pip install nltk

     |████████████████████████████████| 1.1MB 6.4MB/s 
     |████████████████████████████████| 1.1MB 44.1MB/s 
     |████████████████████████████████| 3.0MB 54.8MB/s 
     |████████████████████████████████| 890kB 55.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d740f1cb71c0b391577162d215af3d8733d72d32fdbd11561098d7d3882b2e45
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import io
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import nltk
from sklearn.model_selection import train_test_split
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/text

/content/drive/My Drive/MeasEval-main/data/train/text


In [ ]:
files=os.listdir()

In [ ]:
cd ..

/content/drive/My Drive/MeasEval-main/data/train


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join('text',i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv=os.path.join('tsv',i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)
    
  
  
  
  

In [ ]:
print(len(NERdata))

1225


In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
NERdf.head(5)

,Id,Text,Entity
0,S0012821X13007309-1691,In the Furlo section the CTBI lies within the ...,"[[], [], [], [], [], [], [], [], [], [], [], [..."
1,S0012821X13007309-1691,The Livello Bonarelli is a 1 m thick condensed...,"[[], [], [MeasuredEntity], [MeasuredEntity], [..."
2,S0012821X13007309-1691,Up to 20 m beneath the Bonarelli level there a...,"[[], [Quantity], [Quantity], [Quantity], [Quan..."
3,S0012821X13007309-1691,The δ13Corg record has a narrow variation in b...,"[[], [], [MeasuredEntity], [MeasuredEntity], [..."
4,S0012821X13007309-1691,The characteristic positive excursion in δ13Co...,"[[], [], [MeasuredEntity, MeasuredEntity, Meas..."


In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]
  print(textlabels[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [ ]:
len(textlabels[1])

512

In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
import numpy as np

In [ ]:
train_seq = np.array(inputs['input_ids'])
train_mask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"Quantity":0, "MeasuredEntity": 1, "MeasuredProperty": 2, "Qualifier": 3}

In [ ]:
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),4));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
train_seq, val_seq, train_mask, val_mask, train_y, val_y = train_test_split(train_seq, train_mask, onehot_labels, test_size=0.1, random_state=42)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train_data = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(train_mask),torch.from_numpy(train_y))
val_data = TensorDataset(torch.from_numpy(val_seq), torch.from_numpy(val_mask),torch.from_numpy(val_y))

In [ ]:
batch_size = 38
train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
i=0
for param in model.parameters():
    param.requires_grad = False

In [ ]:
params = list(model.named_parameters())

for p in params:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))



embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)
encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (768,)
encoder.layer.0.attention.output.LayerNorm.weight             (768,)
encoder.layer.0.attention.output.L

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      embed, cls_hs = self.bert(sent_id, attention_mask=mask)
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x

In [ ]:
device = torch.device("cuda")

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,4)
bert_model = bert_model.to(device)

In [ ]:
print(bert_model)

BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [ ]:
lr = 0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(bert_model.parameters(),lr = 0.005)

In [ ]:
epochs = 5
for e in range(epochs):
  
  bert_model.train()
  i=0
  train_loss=0
  for seq, mask, y in train_loader:
    bert_model.zero_grad()
    y_pred = bert_model(seq.to(device), mask.to(device))
    loss = criterion(y_pred,y.to(device).float())
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*batch_size
    if(i%5==0):
      print("Epoch-{}/{} Iterations-{} loss-{}".format(e+1,epochs,i+1,loss.item()))
    i+=1
  
  
  bert_model.eval()
  val_loss=0
  for seq, mask, y in val_loader:
    bert_model.zero_grad()
    y_pred = bert_model(seq.to(device), mask.to(device))
    loss = criterion(y_pred,y.to(device).float())
    val_loss += loss.item()*batch_size
    
    i+=1
  if(e%5==0):
    torch.save(bert_model,'/content/drive/My Drive/Model/bert_{}.pt'.format(e))
  
  print("Epoch-{}/{} train_loss-{} Val_loss-{}".format(e+1,epochs,train_loss/len(train_loader),val_loss/len(val_loader)))
    
  
  
  

Epoch-1/5 Iterations-1 loss-0.003825786989182234
Epoch-1/5 Iterations-6 loss-0.004665067419409752
Epoch-1/5 Iterations-11 loss-0.0032873288728296757
Epoch-1/5 Iterations-16 loss-0.003211055416613817
Epoch-1/5 Iterations-21 loss-0.0024412889033555984
Epoch-1/5 Iterations-26 loss-0.005183529108762741
Epoch-1/5 train_loss-0.12214591423178027 Val_loss-0.19322757178451866
Epoch-2/5 Iterations-1 loss-0.003395254723727703
Epoch-2/5 Iterations-6 loss-0.004970730282366276
Epoch-2/5 Iterations-11 loss-0.002815932733938098
Epoch-2/5 Iterations-16 loss-0.004002842586487532
Epoch-2/5 Iterations-21 loss-0.0024473597295582294
Epoch-2/5 Iterations-26 loss-0.004978496581315994
Epoch-2/5 train_loss-0.12485620146617293 Val_loss-0.23532094131223857
Epoch-3/5 Iterations-1 loss-0.00206369417719543
Epoch-3/5 Iterations-6 loss-0.0029279664158821106
Epoch-3/5 Iterations-11 loss-0.003210482420399785
Epoch-3/5 Iterations-16 loss-0.0031591958831995726
Epoch-3/5 Iterations-21 loss-0.0038703971076756716
Epoch-3/5 I

In [ ]:
pos=0
neg=0
for seq, mask, y in val_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    # for i in range(0, len(np_act[0])):
    #   print(np_act[0][i])
    #   print(np_out[0][i]>=0.3)
    #   print()
    # break
    #print(np.where(np_act==1))
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i][j][k]>=0.3:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0
    
    
    #print(np.where(np_out==1))
    #print("##########")
    #print(np.where(np_act==1))

    # for i in range(np_out.shape[0]):
    #   for j in range(np_out.shape[1]):
    #     if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
    #       if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
    #         pos = pos + 1
    #       else:
    #         neg = neg + 1

    for i in range(np_out.shape[0]):
     for j in range(np_out.shape[1]):
       for k in range(np_out.shape[2]):
         if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
           if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
             pos = pos + 1
           else:
             neg = neg + 1

print(pos/(pos+neg))
    

0.4941935483870968


In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in val_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    #print(np_out)
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i][j][k]>=0.3:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------NER RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.6023054755043228
--------NER RESULTS--------
Accuracy:-0.9984438516260162
Modified Accuracy:-0.4941935483870968
Precision:-0.7408123791102514
Recall:-0.5975039001560063
F1 score:-0.6614853195164075


In [ ]:
# torch.save(bert_model,'/content/drive/My Drive/Model/Bilstm_1_30.pt')

In [ ]:
bert_model=torch.load('/content/drive/My Drive/Model/Bilstm_1_30.pt')

In [ ]:
idx2labels = {0:"Quantity", 1:"MeasuredEntity", 2: "MeasuredProperty", 3: "Qualifier"}

In [ ]:
s = "the boiling point water is 100 celcius"
tokenized_text = tokenizer.tokenize(s)
test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
model.eval()
temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.3

In [ ]:
for i in range(len(tokenized_text)):
  print(tokenized_text[i] + ": ", end='')
  for j in range(4):
    if(temp[0][i+1][j]):
      print(idx2labels[j] + " ", end='')
  print()

the: 
boiling: 
point: MeasuredProperty 
water: MeasuredProperty 
is: 
100: Quantity 
ce: Quantity 
##lc: Quantity 
##ius: Quantity 
